In [1]:
import functools
import pandas as pd
# reset colwitdth options when running all cells 
pd.reset_option('display.max_colwidth')

In [2]:
filename = "tweets_2018_10.json"
tweets = pd.read_json("../data/"+filename)
print("Tweets DF shape: ", tweets.shape)
num_tweets_loaded = tweets.shape[0]
tweets.head()


Tweets DF shape:  (699237, 21)


,has_media,hashtags,img_urls,is_replied,is_reply_to,likes,links,parent_tweet_id,replies,reply_to_users,...,screen_name,text,text_html,timestamp,timestamp_epochs,tweet_id,tweet_url,user_id,username,video_url
0,False,[],[],False,False,0,[],,0,[],...,21Deepakchauhan,“Our senses enable us to perceive only a minut...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2018-05-07 23:59:54,2018-05-07 23:59:54,993641605260238848,/21Deepakchauhan/status/993641605260238848,4211216135,Deepak Chauhan,
1,True,"[Grimes, ElonMusk, Tesla]",[https://pbs.twimg.com/media/Dcofwl6XUAAXo45.jpg],True,False,3,[],,3,[],...,SpartanWire,19 years ago the guy on the left had no chance...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2018-05-07 23:59:51,2018-05-07 23:59:51,993641592488628224,/SpartanWire/status/993641592488628224,3704044761,SpartanWire,
2,False,[],[],False,False,0,[],,0,[],...,everymoshling,Who are these Zoshling creatures? I'm afraid I...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2018-05-07 23:59:18,2018-05-07 23:59:18,993641453556387840,/everymoshling/status/993641453556387840,3719699973,Buster Bumblechops,
3,False,[],[],False,False,0,[https://www.zerohedge.com/news/2018-05-07/aft...,,0,[],...,NosbigsTrader,"After Taunting Shorts With Threats Of ""Unreal ...","<p class=""TweetTextSize js-tweet-text tweet-te...",2018-05-07 23:59:15,2018-05-07 23:59:15,993641444190638080,/NosbigsTrader/status/993641444190638082,747614854073171968,SzeriNosBiggsTrading2.0,
4,False,[],[],False,False,0,[http://dlvr.it/QS5YD0],,0,[],...,Forextrade3,Intraday Update: Dow Jumps 150 Points as Tesla...,"<p class=""TweetTextSize js-tweet-text tweet-te...",2018-05-07 23:59:15,2018-05-07 23:59:15,993641443737550848,/Forextrade3/status/993641443737550848,619674970,Forex trade,


In [3]:
# uncomment lines below to generate a dataframe report
# import pandas_profiling as profile
# profile = profile.ProfileReport(tweets, title='Daily Tweets Profiling Report', html={'style':{'full_width':True}})
# profile.to_file(output_file="tweets02_df_report.html")

In [4]:
# drop columns we will not use
tweets.drop(columns='has_media parent_tweet_id video_url user_id tweet_url tweet_id is_reply_to reply_to_users '
            'timestamp_epochs text_html screen_name is_replied'.split(), inplace=True)

# rearrange columns
cols = 'timestamp hashtags text username likes replies retweets links img_urls'.split() 
tweets = tweets[cols]

print("Tweets DF shape: ", tweets.shape)
tweets.head()

Tweets DF shape:  (699237, 9)


,timestamp,hashtags,text,username,likes,replies,retweets,links,img_urls
0,2018-05-07 23:59:54,[],“Our senses enable us to perceive only a minut...,Deepak Chauhan,0,0,0,[],[]
1,2018-05-07 23:59:51,"[Grimes, ElonMusk, Tesla]",19 years ago the guy on the left had no chance...,SpartanWire,3,3,0,[],[https://pbs.twimg.com/media/Dcofwl6XUAAXo45.jpg]
2,2018-05-07 23:59:18,[],Who are these Zoshling creatures? I'm afraid I...,Buster Bumblechops,0,0,0,[],[]
3,2018-05-07 23:59:15,[],"After Taunting Shorts With Threats Of ""Unreal ...",SzeriNosBiggsTrading2.0,0,0,0,[https://www.zerohedge.com/news/2018-05-07/aft...,[]
4,2018-05-07 23:59:15,[],Intraday Update: Dow Jumps 150 Points as Tesla...,Forex trade,0,0,0,[http://dlvr.it/QS5YD0],[]


In [5]:
## remove all tweets that do not have a tesla or elonmusk hashtag
# check hashtags column types first
# print(tweets['hashtags'].dtypes)

# UPDATE: After discussion, we decided to remain popular tweets 
# even when they do not have a tag, expecting lots of people not always tagging their texts

In [6]:
# getting more familiar with the data types
print(type(tweets.iloc[0]['hashtags']))
print(' '.join(tweets.iloc[1]['hashtags']))
print('tesla' in ' '.join(tweets.iloc[1]['hashtags']))
# type(tweets.iloc[1]['hashtags'][0])

#INSIGHT: Even the dtype of the hashtags column is 'object', 
# when we access it, we get a list

<class 'list'>
Grimes ElonMusk Tesla
False


#### Observation:
We have tweets in different languages: Already saw one in German. Would suggest to first ignore this fact. 

In [7]:
# print some tweets to see how we could further clean them up
print('\n\n'.join(list(tweets.iloc[0:20:5,2])))


“Our senses enable us to perceive only a minute portion of the outside world.”
~Nikola Tesla

Tesla patent reveals secret behind its solar roof tile’s camouflage capacity - Tesla developed a new glass with louvers that reflect that light in a way that makes it look opaque at a lower angle when combined with a backsheet layer while still remaining… http://bit.ly/2jFifev 

Louis V Tesla

Live on the outdoor concert stage @OOBBC Folsom Kraft Kitchen and Biergarten 5/19 from 7-11pm.  @FrankHannon FREE fan appreciation concert and CD release of his new album “From One Place to Another” Vol 2.  Come check out the legendary Tesla guitar player in action! pic.twitter.com/GmtJ2Yw5uG


#### Observations:
- remove Links (can also later be removed by spacy!)
- some tweets do not contain Tesla related information at all 

In [8]:
# remove links from all the tweet texts
tweets['text'] = [functools.reduce(lambda text, link: text.replace(link, ''), links, text) 
                  for (text,links) in zip(tweets['text'],tweets['links']) ]
tweets['text'] = [functools.reduce(lambda text, link: text.replace(link, ''), links, text)
                  for (text,links) in zip(tweets['text'],tweets['img_urls']) ]

print('\n\n'.join(list(tweets.iloc[0:20:5,2])))

“Our senses enable us to perceive only a minute portion of the outside world.”
~Nikola Tesla

Tesla patent reveals secret behind its solar roof tile’s camouflage capacity - Tesla developed a new glass with louvers that reflect that light in a way that makes it look opaque at a lower angle when combined with a backsheet layer while still remaining…  

Louis V Tesla

Live on the outdoor concert stage @OOBBC Folsom Kraft Kitchen and Biergarten 5/19 from 7-11pm.  @FrankHannon FREE fan appreciation concert and CD release of his new album “From One Place to Another” Vol 2.  Come check out the legendary Tesla guitar player in action! pic.twitter.com/GmtJ2Yw5uG


In [9]:
# Some Links could still not be removed as they are not saved as an image url or link. We'll remove them later with spacy.
# remove columns "links" and "img_urls"
tweets.drop(columns=['links', 'img_urls'], inplace=True)
print(tweets.shape)
tweets.head()

(699237, 7)


,timestamp,hashtags,text,username,likes,replies,retweets
0,2018-05-07 23:59:54,[],“Our senses enable us to perceive only a minut...,Deepak Chauhan,0,0,0
1,2018-05-07 23:59:51,"[Grimes, ElonMusk, Tesla]",19 years ago the guy on the left had no chance...,SpartanWire,3,3,0
2,2018-05-07 23:59:18,[],Who are these Zoshling creatures? I'm afraid I...,Buster Bumblechops,0,0,0
3,2018-05-07 23:59:15,[],"After Taunting Shorts With Threats Of ""Unreal ...",SzeriNosBiggsTrading2.0,0,0,0
4,2018-05-07 23:59:15,[],Intraday Update: Dow Jumps 150 Points as Tesla...,Forex trade,0,0,0


### Discussion:
Actually we don't think that tweets that were not liked and retweeted could possibly influence the stock market. So we will remove them. On the other hand, a tweet could be a reaction to a change in the stock price, don't get any attention (no likes and retweets) but still relevant!

In [10]:
# an important tweet will have at least one retweet (info spreading)
# and several likes (attention, sentiment) or replies (discussion)
best_tweets = tweets[(tweets['retweets']>0) | (tweets['likes']>1) | (tweets['replies']>1)]
print(best_tweets.shape)

# display whole tweet texts
pd.set_option('display.max_colwidth', -1)
best_tweets.head()

(136478, 7)
/home/jonas/Documents/NLP4Web/.venv/lib64/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  import sys


,timestamp,hashtags,text,username,likes,replies,retweets
1,2018-05-07 23:59:51,"[Grimes, ElonMusk, Tesla]",19 years ago the guy on the left had no chance at #Grimes.\n#ElonMusk #Tesla pic.twitter.com/Y4uW6c9gyc,SpartanWire,3,3,0
12,2018-05-07 23:57:37,[],2 for 1 hotbox the Tesla be grand,roB leixLip,1,2,0
15,2018-05-07 23:57:17,[],Live on the outdoor concert stage @OOBBC Folsom Kraft Kitchen and Biergarten 5/19 from 7-11pm. @FrankHannon FREE fan appreciation concert and CD release of his new album “From One Place to Another” Vol 2. Come check out the legendary Tesla guitar player in action! pic.twitter.com/GmtJ2Yw5uG,OutOfBoundsBrewing,3,0,0
18,2018-05-07 23:55:33,[],He's going to insert a single Tesla matchbox car into the pussy and make motor noises. Revolutionary.,Rev. Dr. Jacuzzi Toblerone,4,1,0
21,2018-05-07 23:59:51,"[Grimes, ElonMusk, Tesla]",19 years ago the guy on the left had no chance at #Grimes.\n#ElonMusk #Tesla pic.twitter.com/Y4uW6c9gyc,SpartanWire,3,3,0


In [11]:
# remove tweets with too many hashtags, where 'tesla' is just a single one of 
# and looses its relevance this way

# check such kind of tweets first, if they're really irrelevant
too_many_tags = best_tweets[best_tweets['hashtags'].apply(lambda tags: len(tags)>10)==True]

print(too_many_tags.shape)
too_many_tags.head()

(2023, 7)


,timestamp,hashtags,text,username,likes,replies,retweets
996,2018-03-18 23:49:05,"[NikolaTesla, Inventor, MiningEngineer, ArmillarySphere, map, mine, globe, DeathRay, generators, turbines, drilling, towers, watertable, boreholes, world, earthcrust, mineraldeposits, Tesla, GoldWar, Conspiracypic]",#NikolaTesla #Inventor/#MiningEngineer used #ArmillarySphere to #map & #mine #globe;#DeathRay #generators #turbines #drilling #towers & covert specs note #watertable & #boreholes depth of 300'-traveled #world punctured #earthcrust for #mineraldeposits?\n#Tesla #GoldWar #Conspiracypic.twitter.com/olpjNzfafR,Ancient Investigations,2,0,2
1014,2018-03-18 23:44:45,"[tesla, siemens, astronauts, medical, hospital, nasa, medicaldevice, tech, SiliconValleyHBO, Healthcare, space, astrophysics, doctor, nurses]",In the future #tesla will buy #siemens to use its medical devices for #astronauts on long trips to and from mars...(tgj) #medical #hospital #nasa #medicaldevice #tech #SiliconValleyHBO #Healthcare #space #astrophysics #doctor #nurses,the ghetto joker,1,0,1
1036,2018-03-18 23:36:51,"[Nokia, Ericsson, IBM, Motorola, Samsung, HP, Sony, Comcast, Intel, Microsoft, Apple, Dell, Amazon, Netflix, Google, Salesforce, Tesla, Facebook, Twitter, Airbnb, Uber]",1865 #Nokia\n1876 #Ericsson \n1911 #IBM\n1928 #Motorola\n1938 #Samsung\n1939 #HP\n1946 #Sony\n1963 #Comcast\n1968 #Intel\n1975 #Microsoft\n1976 #Apple\n1984 #Dell\n1994 #Amazon\n1997 #Netflix\n1998 #Google\n1999 #Salesforce\n2003 #Tesla\n2004 #Facebook\n2006 #Twitter\n2008 #Airbnb\n2009 #Uber,Vanessa ₿,33,4,6
1079,2018-03-18 23:25:20,"[ASX, dow, finance, Cobalt, Tesla, trump, NYSE, Steel, ElectricCar, rt, ausbiz]",$GPP 2M CR will be used to advance the Turesi Ridge drilling campaign at the Morabisi # Lithium project \n14 x 2.2Li samples \n#ASX #dow #finance $FMG #Cobalt $SQM #Tesla #trump #NYSE $CBA $ANZ $NAB $KDR $GXY $PLS $TLS #Steel #ElectricCar #rt #ausbiz\n pic.twitter.com/JPXykbQ0Ur,The Sower🌾,0,1,1
1080,2018-03-18 23:25:14,"[ASX, dow, finance, Cobalt, Tesla, trump, NYSE, Steel, ElectricCar, rt, ausbiz]",$GPP 2M CR will be used to advance the Turesi Ridge drilling campaign at the Morabisi # Lithium project \n14 x 2.2Li samples \n#ASX #dow #finance $FMG #Cobalt $SQM #Tesla #trump #NYSE $CBA $ANZ $NAB $KDR $GXY $PLS $TLS #Steel #ElectricCar #rt #ausbiz\n pic.twitter.com/bOdsN03viZ,SeeThru👁,0,0,1


In [12]:
# some tweets with many tags seem to be quite relevant, 
# so just remove those coming from specific users
remove_users = ['ADRIAN AGUILAR', 'Our Creator', 'BlindFace', 'Bringitorc']

print(best_tweets.shape)
best_tweets = best_tweets[best_tweets['username'].apply(lambda user: user in remove_users)==False]

# do remove tweets with too many tags
best_tweets = best_tweets[best_tweets['hashtags'].apply(lambda tags: len(tags)>10)==False]
print(best_tweets.shape)
best_tweets.head()


(136478, 7)
(134452, 7)


,timestamp,hashtags,text,username,likes,replies,retweets
1,2018-05-07 23:59:51,"[Grimes, ElonMusk, Tesla]",19 years ago the guy on the left had no chance at #Grimes.\n#ElonMusk #Tesla pic.twitter.com/Y4uW6c9gyc,SpartanWire,3,3,0
12,2018-05-07 23:57:37,[],2 for 1 hotbox the Tesla be grand,roB leixLip,1,2,0
15,2018-05-07 23:57:17,[],Live on the outdoor concert stage @OOBBC Folsom Kraft Kitchen and Biergarten 5/19 from 7-11pm. @FrankHannon FREE fan appreciation concert and CD release of his new album “From One Place to Another” Vol 2. Come check out the legendary Tesla guitar player in action! pic.twitter.com/GmtJ2Yw5uG,OutOfBoundsBrewing,3,0,0
18,2018-05-07 23:55:33,[],He's going to insert a single Tesla matchbox car into the pussy and make motor noises. Revolutionary.,Rev. Dr. Jacuzzi Toblerone,4,1,0
21,2018-05-07 23:59:51,"[Grimes, ElonMusk, Tesla]",19 years ago the guy on the left had no chance at #Grimes.\n#ElonMusk #Tesla pic.twitter.com/Y4uW6c9gyc,SpartanWire,3,3,0


### Observation:
Even after doing quite a lot of preprocessing, we still see some tweets that don't for sure contain relevant information. In the scope of this project, we stop the data cleaning at this moment, but would definitely do much more in a real world scenario, including collecting much more tweets.


In [13]:
SAVE_DF = True
if SAVE_DF:
    num_tweets_cleaned = best_tweets.shape[0]
    best_tweets.to_json('../data/processed/less_strict/'+filename)
    print('FINISHED SAVING {0} ({1:.0f}%) tweets!'.format(num_tweets_cleaned, 100*num_tweets_cleaned/num_tweets_loaded))

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/less_strict/tweets_2018_10.json'